In [ ]:
import cv2
from ultralytics import YOLO
from paddleocr import PaddleOCR
import os
import numpy as np

image_path = '/mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/YOLOv8/mainali.jpg'
model_path = "/mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/pretrained models/Citizenship_bestmodel.pt"

model = YOLO(model_path)

results = model(image_path)

image = cv2.imread(image_path)

ocr = PaddleOCR(use_angle_cls=True, lang='ne')  # 'ne' is the language code for Nepali

class_map = {
    0: 'citizenship_number',
    1: 'day',
    2: 'district',
    3: 'document_type',
    4: 'father_name',
    5: 'gender',
    6: 'month',
    7: 'municipality',
    8: 'name',
    9: 'ward',
    10: 'year'
}

def get_color(class_id):
    np.random.seed(class_id)  # Seed for reproducibility
    return tuple(np.random.randint(0, 255, 3).tolist())

base_name = os.path.splitext(os.path.basename(image_path))[0]

output_dir = 'OCR_Outputs'
os.makedirs(output_dir, exist_ok=True)

output_file_path = os.path.join(output_dir, f'{base_name}_results.txt')

with open(output_file_path, 'w') as file:
    for result in results:
        boxes = result.boxes.xyxy
        class_ids = result.boxes.cls
        for box, class_id in zip(boxes, class_ids):
            x1, y1, x2, y2 = map(int, box.tolist())
            
            cropped_img = image[y1:y2, x1:x2]

            ocr_result = ocr.ocr(cropped_img)

            for line in ocr_result:
                for text in line:
                    file.write(f"Class: {class_map.get(int(class_id), 'Unknown')}, Text: {text[1][0]}\n")
                    
            color = get_color(int(class_id))
            label = class_map.get(int(class_id), 'Unknown')
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

annotated_image_path = os.path.join(output_dir, f'{base_name}_annotated.jpg')
cv2.imwrite(annotated_image_path, image)
